In [ ]:
# this is intended for the Group project

import os
import pandas as pd
import numpy as np
from scipy import stats
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier
from sklearn.model_selection import train_test_split, cross_val_score,GridSearchCV
from sklearn.metrics import accuracy_score

import matplotlib.pyplot as plt

# New Section

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
IBM_price = pd.read_csv("/content/IMB_revisit - MacroTrends_Data_Download_IBM.csv")
IBM_price.columns = ['date', 'IBM_open', 'IBM_high', 'IBM_low', 'IBM_close', 'IBM_volume']
IBM_price = IBM_price[(IBM_price['date'] > '2011-12-31')]
IBM_price.head()

,date,IBM_open,IBM_high,IBM_low,IBM_close,IBM_volume
12588,2012-01-03,124.5256,125.8460,124.0388,124.2388,5646000
12589,2012-01-04,123.7520,124.2589,123.3319,123.7320,4346700
12590,2012-01-05,123.2452,123.3919,122.1048,123.1452,4463100
12591,2012-01-06,122.9651,123.0251,121.5780,121.7314,4897100
12592,2012-01-09,121.5047,121.5513,120.2176,121.0979,5201200


In [ ]:
NASDAQ = pd.read_csv("/content/NASDAQ index.csv")
NASDAQ.columns = ['date', 'NASDAQ_open', 'NASDAQ_high', 'NASDAQ_low', 'NASDAQ_close', 'NASDAQ_Adj_close', 'NASDAQ_volume']
NASDAQ = NASDAQ[(NASDAQ['date'] > '2011-12-31')]
NASDAQ.head()

,date,NASDAQ_open,NASDAQ_high,NASDAQ_low,NASDAQ_close,NASDAQ_Adj_close,NASDAQ_volume
5547,2012-01-03,2657.389893,2665.899902,2641.979980,2648.719971,2648.719971,1636850000
5548,2012-01-04,2639.899902,2653.179932,2627.229980,2648.360107,2648.360107,1670530000
5549,2012-01-05,2642.570068,2673.560059,2631.229980,2669.860107,2669.860107,1836410000
5550,2012-01-06,2671.169922,2682.120117,2658.830078,2674.219971,2674.219971,1683090000
5551,2012-01-09,2682.979980,2683.780029,2662.959961,2676.560059,2676.560059,1768080000


In [ ]:
SP = pd.read_csv("/content/S_P data.csv")
SP.head()

,Date,Close/Last,Volume,Open,High,Low
0,11/16/2021,4700.90,--,4679.42,4714.95,4679.42
1,11/15/2021,4682.80,--,4689.30,4697.42,4672.86
2,11/12/2021,4682.85,--,4655.24,4688.47,4650.77
3,11/11/2021,4649.27,--,4659.39,4664.55,4648.31
4,11/10/2021,4646.71,--,4670.26,4684.85,4630.86


In [ ]:
from datetime import datetime
SP['Date'] = SP['Date'].astype('string')
for i in range(len(SP)):
  SP.iloc[i,0] = datetime.strptime(SP.iloc[i,0], "%m/%d/%Y").strftime("%Y-%m-%d")

In [ ]:
SP.columns = ['date', 'SP_close', 'SP_Volume', 'SP_Open', 'SP_High', 'SP_low']
SP = SP[(SP['date'] > '2011-12-31')]
SP.tail()

,date,SP_close,SP_Volume,SP_Open,SP_High,SP_low
2519,2012-01-06,1277.81,--,1280.93,1281.84,1273.34
2520,2012-01-05,1281.06,--,1277.30,1283.05,1265.26
2521,2012-01-04,1277.30,--,1277.03,1278.73,1268.10
2522,2012-01-03,1277.06,--,1258.86,1284.62,1258.86
2523,2012-01-02,1257.60,--,0.00,0.00,0.00


In [ ]:
inflation = pd.read_csv("/content/T10YIE.csv")
inflation.columns = ['date', 'interest_rate']
inflation = inflation[(inflation['date'] > '2011-12-31')]
inflation.head()

,date,interest_rate
33,2012-01-02,.
34,2012-01-03,2.01
35,2012-01-04,2.08
36,2012-01-05,2.12
37,2012-01-06,2.09


In [ ]:
#gdp = pd.read_csv("/content/GDP.xlsx - Sheet1.csv")
gdp = pd.read_csv("/content/GDP.csv")
display(gdp)
gdp.columns = ['date', 'gdp']
gdp = gdp[(gdp['date'] > '2011-12-31')]
gdp.head()

,DATE,GDP
0,1990-01-01,5872.701
1,1990-04-01,5960.028
2,1990-07-01,6015.116
3,1990-10-01,6004.733
4,1991-01-01,6035.178
...,...,...
122,2020-07-01,21138.574
123,2020-10-01,21477.597
124,2021-01-01,22038.226
125,2021-04-01,22740.959


,date,gdp
88,2012-01-01,16068.824
89,2012-04-01,16207.130
90,2012-07-01,16319.540
91,2012-10-01,16420.386
92,2013-01-01,16629.050


In [ ]:
merge_IBM_NAS = pd.merge(IBM_price, NASDAQ, on= 'date', how= 'outer')

In [ ]:
merge_one_SP = pd.merge(merge_IBM_NAS, SP, on= 'date', how= 'outer')

In [ ]:
merge_two_inflation = pd.merge(merge_one_SP, inflation,on= 'date', how= 'outer')

In [ ]:
merge_three_gdp = pd.merge(merge_two_inflation, gdp,on= 'date', how= 'outer')

In [ ]:
merge_three_gdp = merge_three_gdp.sort_values(['date'])
merge_three_gdp.head()

,date,IBM_open,IBM_high,IBM_low,IBM_close,IBM_volume,NASDAQ_open,NASDAQ_high,NASDAQ_low,NASDAQ_close,NASDAQ_Adj_close,NASDAQ_volume,SP_close,SP_Volume,SP_Open,SP_High,SP_low,interest_rate,gdp
2577,2012-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16068.824
2523,2012-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1257.60,--,0.00,0.00,0.00,.,NaN
0,2012-01-03,124.5256,125.8460,124.0388,124.2388,5646000.0,2657.389893,2665.899902,2641.97998,2648.719971,2648.719971,1.636850e+09,1277.06,--,1258.86,1284.62,1258.86,2.01,NaN
1,2012-01-04,123.7520,124.2589,123.3319,123.7320,4346700.0,2639.899902,2653.179932,2627.22998,2648.360107,2648.360107,1.670530e+09,1277.30,--,1277.03,1278.73,1268.10,2.08,NaN
2,2012-01-05,123.2452,123.3919,122.1048,123.1452,4463100.0,2642.570068,2673.560059,2631.22998,2669.860107,2669.860107,1.836410e+09,1281.06,--,1277.30,1283.05,1265.26,2.12,NaN


In [ ]:
yield1 = pd.read_csv('/content/yield_curves.csv')

FileNotFoundError: ignored

In [ ]:
yield1.columns = ['date', 'yield_rate']

In [ ]:
final_merge = pd.merge(merge_three_gdp, yield1 ,on= 'date', how= 'outer')

In [ ]:
final_merge.head()

In [ ]:
final_merge.to_csv('final_dataset_stocks.csv')